In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
cik = '796343'
def download_txt(cik):
    base = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb=&owner=include&count=100&search_text='.format(cik, '8-k')
    heads = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

    content = requests.get(base, headers = heads)
    bs4_base = BeautifulSoup(content.text)

    if not os.path.exists('8k-forms/'+cik):
        os.makedirs('8k-forms/'+cik)
    
    year = ""
    for hrefs in bs4_base.find_all('a', href=True):
        
        if hrefs['href'].split('.')[-1] == 'htm' and len(hrefs['href'].split('/')) == 7:
            if hrefs.parent.parent.td.get_text().lower() != '8-k':
                continue
            date = hrefs.parent.parent.find_all('td')[3].get_text()
            z = date[:4]
            if year == "":
                year = z
            htm_url = 'https://www.sec.gov/' + hrefs['href']
            content = requests.get(htm_url, headers = heads)
            bs4_htm = BeautifulSoup(content.text)

            if year == z or z == '2021':
                for htm_hrefs in bs4_htm.find_all('a', href=True):
                    if htm_hrefs['href'].split('.')[-1] == 'txt':
                        url = 'https://www.sec.gov/' + htm_hrefs['href']

                        request_text = requests.get(url, headers = heads).content
                        print(z)
                        
                        bs4_end = BeautifulSoup(request_text)
                        
                        file_path = '8k-forms/' + cik + '/' + date + '.txt'

                        with open(file_path, "w", encoding="utf-8") as file:
                            file.write(str(bs4_end))


In [ ]:

download_txt(cik)



In [ ]:
df = pd.read_csv('base_comp_cik2.csv')
df = df[df['CIK'].isna() == False]

In [ ]:
all_paths = {}
for index, i in enumerate(df.index):
    cik = str(int(df.iloc[i]['CIK']))
    try:
        download_txt(cik)
    except:
        pass
    print(index, i, cik)
        

In [ ]:
all_paths

In [ ]:
len(df)

In [ ]:
!conda install ipykernel --update-deps --force-reinstall